# Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

## Part 1.
**Formulate the least square problem.**
$$\min_{A_{12}, A_{21}} \sum_{i=1}^{n} (p(x^{(i)}, A_{12},A_{21})-p^{(i)})^2  \quad \forall i=1,2,...11$$

such that  $$p(x^{(i)}, A_{12},A_{21})=x^{(i)}_1\exp\left(A_{12}\left(\frac{A_{21}x^{(i)}_2}{A_{12}x^{(i)}_1+A_{21}x^{(i)}_2}\right)^2\right)p_{water}^{sat} + x_2\exp\left(A_{21}\left(\frac{A_{12}x^{(i)}_1}{A_{12}x^{(i)}_1+A_{21}x^{(i)}_2}\right)^2\right)p_{1,4 dioxane}^{sat} $$
and $$ x_2 = 1-x_1 $$

## Part 2. 
**Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1.** <br>
We will calculate the saturation pressures, using $p^{sat}=10^{a_1-\frac{a_2}{T+a_3}}$.

In [7]:
# HOUSEKEEPIN'
import torch as t
from torch.autograd import Variable
import numpy as np

p_satw=10**(8.071 - 1730.63/(20+233.426)) # 17.460784103526855 
p_sat14=10**(7.43155 - 1554.679/(20+240.337)) # 28.824099527405245   
x=[ 0.0 , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9 , 1.0 ]
p=[ 28.1 , 34.4 , 36.7 , 36.9 , 36.8 , 36.7 , 36.5 , 35.4 , 32.9 , 27.7 , 17.5 ]

print(p_satw)
print(p_sat14)

17.460784103526855
28.824099527405245


In [8]:
# The FUNCTIONS.

def press(a,xi): # Calculating pressure!!
    x1=xi
    x2=1-x1
    A12= a[0]
    A21= a[1]
    return x1 * t.exp(A12* ( (A21*x2)/(A12*x1 + A21*x2) )**2 ) * p_satw + x2 * t.exp(A21* ( (A12*x1)/(A12*x1 + A21*x2) )**2 ) * p_sat14

def objf(a): # Total sum objective function...
    total=0
    for i in range(len(x)):
        xi=x[i]
        pres=press(a,xi)
        #print(press)
        total = total + (pres-p[i])**2  # Sum the squared difference.
    return total

def lines(a):  # because you just gotta have a line search!
    step=.1 # initiate
    while objf(a-step*a.grad) > objf(a)-step*(0)*np.matmul(a.grad,a.grad):
        step=.5*step
    return step

# VARIABLE SETUP.

a = Variable(t.tensor([1.0, 6]), requires_grad=True) 
e = 100 # Also our error term! Initialize.

# RUN THIS THING!
while e > 0.1:  # Error stop criteria. 
    objective=objf(a)
    objective.backward()
    step=lines(a)
    #print('Objective func is now ' +str(objective.data.numpy()))
    e = t.linalg.norm(a.grad) # update the error value.
    with t.no_grad():
        #print('a is ' + str(a))
        #print('grad is ' + str(a.grad))
        a -= step * a.grad
        #print('a is now ' + str(a) + ' and e is '+str(e))
        #print('step')
        # need to clear the gradient at every step, or otherwise it will accumulate...
        a.grad.zero_()
        
print('Final a is now ' + str(a.data.numpy()))
print('The objective function is now ' + str(objective.data.numpy()))

Final a is now [1.9594922 1.6899538]
The objective function is now 0.6712008


## Part 3. 
**Compare your optimized model with the data. Does your model fit well with the data?**

In [9]:
# Well, time to calculate these new p values.

a_solve=a.data

print('Data Pressures   Model Pressures')
for i in range(len(x)):
    print(str(p[i]) + '             ' + str(press(a_solve,x[i]).item()))
    

Data Pressures   Model Pressures
28.1             28.824098587036133
34.4             34.64545822143555
36.7             36.45291519165039
36.9             36.86630630493164
36.8             36.87278366088867
36.7             36.749027252197266
36.5             36.39037322998047
35.4             35.3852653503418
32.9             32.9476203918457
27.7             27.72649383544922
17.5             17.460784912109375


The model-generated pressures fit very well ($\pm 0.75$) with the values in the original data. 

---

# Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).

In [2]:
## USING THE TUTORIAL CODE...

# Housekeeping. 

import numpy as np
import sklearn.gaussian_process as gp

from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.datasets import make_classification

# EXPECTED IMPROVEMENT function.

def expected_improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):
    
    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0

    return -1 * expected_improvement

# SAMPLE NEXT HYPERPARAMETER function.

def sample_next_hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):
    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))

        if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x
     

#%% And then finally, the BAYESIAN OPTIMIZATION function.

def bayesian_optimisation(n_iters, sample_loss, bounds, x0=None, n_pre_samples=5,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7):
   
    x_list = []
    y_list = []

    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(sample_loss(params))
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(sample_loss(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for n in range(n_iters):

        model.fit(xp, yp)

        # Sample next hyperparameter
        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * expected_improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = sample_next_hyperparameter(expected_improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        # Sample loss for new set of parameters
        cv_score = sample_loss(next_sample)

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

# Our loss function. AKA OBJECTIVE FUNCTION!!!!
def sample_loss(x): # takes in a vector [x1,x2]
    x1=x[0]
    x2=x[1]
    return -1*((4 - 2.1*x1**2 + (x1**4)/3)*x1**2 + x1*x2 + (-4 + 4*(x2**2))*x2**2)

In [3]:
# Start runnin...'
x_1 = np.linspace(-3,3)
x_2 = np.linspace(-2,2)

# We need the cartesian combination of these two vectors
param_grid = np.array([[x1i, x2i] for x1i in x_1 for x2i in x_2])

real_loss = [sample_loss(params) for params in param_grid]

# The maximum is at:
print('The true minimum value of ' + str(-np.amax(real_loss)) +' is at '+ str(param_grid[np.array(real_loss).argmax(), :]))

The true minimum value of -1.02614400718987 is at [-0.06122449  0.69387755]


In [4]:
import warnings
warnings.filterwarnings('ignore')  # Let's ignore the warnings.

bounds = np.array([[-3, 3], [-2, 2]])

print('running Bayesian Opt.')

xp, yp = bayesian_optimisation(n_iters=100, # HERE WE GO! 
                               sample_loss=sample_loss, 
                               bounds=bounds,
                               n_pre_samples=3,
                               random_search=100000)

print('Out of 100 iterations, the minimum value of ' + str(-np.amax(yp)) +' is at ' + str(xp[np.where(yp == np.amax(yp))[0][0]]))

running Bayesian Opt.
Out of 100 iterations, the minimum value of -1.0314392694751924 is at [ 0.08347011 -0.71028762]
